In [95]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numba import jit

# 1a

In [96]:
plummer_data_1a = pd.read_csv('plummer_regular.csv')
r = plummer_data_1a.loc[:,['r_x', 'r_y', 'r_z']]
r = r.to_numpy()
v = plummer_data_1a.loc[:,['v_x', 'v_y', 'v_z']]
v = v.to_numpy()
mass = np.full((10000,1), 2)

In [97]:
def getAcc(pos : np.ndarray, mass : np.ndarray, G : float, softening : float):
    """
    Calculate the acceleration on each particle due to Newton's Law
    pos  is an N x 3 matrix of positions
    mass is an N x 1 vector of masses
    G is Newton's Gravitational constant
    softening is the softening length
    a is N x 3 matrix of accelerations
    """
    # positions r = [x,y,z] for all particles
    x = pos[:,0:1]
    y = pos[:,1:2]
    z = pos[:,2:3]
    
    # matrix that stores all pairwise particle separations: r_j - r_i
    dx = x.T - x
    dy = y.T - y
    dz = z.T - z
    
    # matrix that stores 1/r^3 for all particle pairwise particle separations 
    inv_r3 = (dx**2 + dy**2 + dz**2 + softening**2)**(-1.5)
    
    ax = G * (dx * inv_r3) @ mass
    ay = G * (dy * inv_r3) @ mass
    az = G * (dz * inv_r3) @ mass
    
    # pack together the acceleration components
    a = np.hstack((ax,ay,az))

    return a

def leapfrog(pos: np.ndarray, vec: np.ndarray, mass: float,G: float, softening: float, dt: float):

    # leapfrog for N-body
    vec = vec + 0.5*dt*getAcc(pos, mass, G, softening)
    pos = pos+ vec*dt
    vec = vec+ 0.5*dt*getAcc(pos, mass, G, softening)

    return pos, vec

In [100]:
pos, vec = leapfrog(r, v, mass, 1, 5, 1)

[[ 73.90581349 -23.07534889   9.07323374]
 [ 33.02005186  32.95930148 -28.77900128]
 [ -3.18853095  75.3347163   54.99117777]
 ...
 [ 46.33014793  62.65750984  44.28870595]
 [ 43.45869855  37.49065256   4.98877903]
 [-27.35344326   3.55654026  45.04195978]] [[ 74.39439288 -23.21583041   9.14930357]
 [ 32.90802957  32.87515952 -28.64411795]
 [ -3.22496335  76.20297651  55.61431647]
 ...
 [ 46.7924918   63.26634437  44.76213142]
 [ 43.42474921  37.48127469   5.02091542]
 [-32.21311019   4.2662703   53.18645717]]


In [99]:
# x = np.array([3,27,5.3]).reshape(3,1)
# print(x.T-x)